<a href="https://colab.research.google.com/github/KimRass/Programming/blob/master/Data%20Science/Machine%20Learning/NLP/NSMC%20%26%20KoBERT%20(Sentiment%20Analysis).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Data Science/Machine Learning/NLP/NSMC & KoBERT (Sentiment Analysis).ipynb
!pip install transformers==4.4.2
!pip install tensorflow_addons
!pip install sentencepiece

import urllib.request
urllib.request.urlretrieve("https://raw.githubusercontent.com/monologg/KoBERT-Transformers/master/kobert_transformers/tokenization_kobert.py", filename="tokenization_kobert.py")

     |████████████████████████████████| 2.0 MB 5.5 MB/s 
     |████████████████████████████████| 895 kB 33.7 MB/s 
     |████████████████████████████████| 3.3 MB 46.7 MB/s 
     |████████████████████████████████| 1.1 MB 5.3 MB/s 
     |████████████████████████████████| 1.2 MB 5.2 MB/s 


('tokenization_kobert.py', <http.client.HTTPMessage at 0x7fb40c5b1e10>)

In [3]:
from google.colab import drive
import os
drive.mount("/content/drive")
os.chdir("/content")

import tensorflow as tf
from tensorflow.keras import Input, Model, Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Layer, Dense, Flatten, Dropout, Concatenate, Add, Dot, Multiply, Reshape, Activation, BatchNormalization, LayerNormalization, SimpleRNNCell, RNN, SimpleRNN, LSTM, Embedding, Bidirectional, TimeDistributed, Conv1D, Conv1DTranspose, Conv2D, Conv2DTranspose, MaxPool1D, MaxPool2D, GlobalMaxPool1D, GlobalMaxPool2D, AveragePooling1D, AveragePooling2D, GlobalAveragePooling1D, GlobalAveragePooling2D, ZeroPadding2D, RepeatVector
from tensorflow.keras.layers.experimental.preprocessing import Rescaling
from tensorflow.keras.utils import to_categorical, plot_model
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator
from tensorflow.keras.optimizers import SGD, Adagrad, Adam
from tensorflow.keras.optimizers.schedules import LearningRateSchedule
# MeanSquaredError, MeanAbsoluteError, MeanAbsolutePercentageError, BinaryCrossentropy, CategoricalCrossentropy, SparseCategoricalCrossentropy, CosineSimilarity
from tensorflow.keras import losses
# MeanSquaredError, RootMeanSquaredError, MeanAbsoluteError, MeanAbsolutePercentageError, BinaryCrossentropy, CategoricalCrossentropy, SparseCategoricalCrossentropy, CosineSimilarity
from tensorflow.keras import metrics
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.activations import linear, sigmoid, relu
from tensorflow.keras.initializers import RandomNormal, glorot_uniform, he_uniform, Constant
import tensorflow_addons as tfa
import numpy as np
import pandas as pd
from transformers import TFBertModel
from tokenization_kobert import KoBertTokenizer
import json
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
import os
import sklearn
# import logging

# logging.basicConfig(level=logging.ERROR)

Mounted at /content/drive


In [4]:
os.chdir("/content/drive/MyDrive/NLP")
ratings_tr = pd.read_table("./Datasets/Naver Sentiment Movie Corpus/ratings_train.txt")
ratings_te = pd.read_table("./Datasets/Naver Sentiment Movie Corpus/ratings_test.txt")

ratings_tr = ratings_tr.dropna(subset=["document"])
ratings_te = ratings_te.dropna(subset=["document"])

In [6]:
tokenizer = KoBertTokenizer.from_pretrained("monologg/kobert")

Downloading:   0%|          | 0.00/371k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/77.8k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/51.0 [00:00<?, ?B/s]

In [7]:
seq_len = 64
def create_bert_embeddings(df):
    df["document"] = df["document"].astype("str")
    df["label"] = df["label"].astype("int")
    
    token_embs = list()
    segment_embs = list()
    pos_embs = list()
    labels = list()
    for _, row in tqdm(df.iterrows()):
        ids = tokenizer.encode(row["document"], max_length=seq_len, truncation=True, padding="max_length")
        n_pads = ids.count(1)
        segment = [0]*seq_len
        pos = [1]*(seq_len - n_pads) + [0]*n_pads

        token_embs.append(ids)
        segment_embs.append(segment)
        pos_embs.append(pos)
        labels.append(row["label"])
    token_embs = np.array(token_embs)
    segment_embs = np.array(segment_embs)
    pos_embs = np.array(pos_embs)
    labels = np.array(labels)
    return [token_embs, pos_embs, segment_embs], labels

tr_X, tr_y = create_bert_embeddings(ratings_tr)

0it [00:00, ?it/s]

# Modeling

In [8]:
def create_model():
    model = TFBertModel.from_pretrained("monologg/kobert", from_pt=True)

    inputs_token = Input(shape=(seq_len,), dtype=tf.int32, name="Input_token")
    inputs_pos = Input(shape=(seq_len,), dtype=tf.int32, name="Input_position")
    inputs_segment = Input(shape=(seq_len,), dtype=tf.int32, name="Input_segment")
    inputs = [inputs_token, inputs_pos, inputs_segment]

    outputs = model(inputs)
    # output[0]은 모든 step의 hidden state. => Many-to-many 문제
    # output[1]은 첫번째 위치의 hidden state. => Many-to-one 문제
    z = Dropout(rate=0.5, name="Dropout")(outputs[1])
    
    outputs = Dense(units=1, activation="sigmoid", kernel_initializer=tf.keras.initializers.TruncatedNormal(stddev=0.02), name="Dense")(z)
    bert_model = Model(inputs=inputs, outputs=outputs)
    bert_model.summary()

    # `total_steps`: batch_size*n_epochs
    optimizer = tfa.optimizers.RectifiedAdam(lr=5.0e-5, total_steps=2344*4, warmup_proportion=0.1, min_lr=1e-5, epsilon=1e-08, clipnorm=1.0)
    bert_model.compile(optimizer=optimizer, loss="binary_crossentropy", metrics=["acc"])
    return bert_model

In [9]:
model_path = "nsmc_kobert.h5"
if os.path.exists(model_path):
    bert_model = create_model()
    bert_model.load_weights(model_path)
else:
    bert_model = create_model()
    hist = bert_model.fit(x=tr_X, y=tr_y, validation_split=0.2, batch_size=64, epochs=3, verbose=1, shuffle=True)
    bert_model.save_weights(model_path, overwrite=True)

Downloading:   0%|          | 0.00/426 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/369M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFBertModel.

All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experi

/usr/local/lib/python3.7/dist-packages/tensorflow_addons/optimizers/rectified_adam.py:120: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


# Evaluation

In [90]:
def create_bert_embeddings_inference(sent):
    ids = tokenizer.encode(sent, max_length=seq_len, truncation=True, padding="max_length")
    n_pads = ids.count(1)
    segment = [0]*seq_len
    pos = [1]*(seq_len - n_pads) + [0]*n_pads

    token_embs = np.array([ids])
    segment_embs = np.array([segment])
    pos_embs = np.array([pos])
    
    return [token_embs, pos_embs, segment_embs]

# Inference

In [91]:
def inference(sent):
    embs = create_bert_embeddings_inference(sent)
    y_pred = bert_model.predict(embs)
    y_pred = np.ravel(y_pred)
    predict_answer = np.round(y_pred, 0).item()
    
    if predict_answer == 0:
      print(f"부정 확률: {1 - y_pred[0]:.1%}")
    elif predict_answer == 1:
      print(f"긍정 확률: {y_pred[0]:.1%}")

In [95]:
inference("좋아하는 배우가 나온다는 점만 좋습니다.")

부정 확률: 99.3%
